In [34]:
import h5py
import numpy as np
import datetime
import inspect
from typing import Any, Dict
import pandas as pd



def recursively_load_dict_contents_from_group(h5_file: h5py.File, path: str) -> Dict[str, Any]:
    """
    Recursively load a HDF5 file into a dictionary

    :param h5_file: the h5py file object
    :param path: the path to the group in the hdf5 file
    :return: a dictionary containing the contents of the hdf5 file
    """

    output = dict()
    for key, item in h5_file[path].items():
        if isinstance(item, h5py.Dataset):
            output[key] = decode_from_hdf5(item[()])
        elif isinstance(item, h5py.Group):
            output[key] = recursively_load_dict_contents_from_group(
                h5_file, path + key + "/"
            )
    return output


def recursively_save_dict_contents_to_group(h5_file: h5py.File, path: str, dic: Dict[str, Any]) -> None:
    """
    Recursively save a dictionary to a HDF5 group

    :param h5_file: h5py.File
        Open HDF5 file
    :param path: str
        Path inside the HDF5 file
    :param dic: dict
        The dictionary containing the data
    """
    for key, item in dic.items():
        item = encode_for_hdf5(key, item)
        if isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5_file, path + key + "/", item)
        else:
            h5_file[path + key] = item


def decode_from_hdf5(item) -> Any:
    """
    Decode an item from HDF5 format to python type.

    This currently just converts __none__ to None and some arrays to lists

    :param item: object
        Item to be decoded

    :return  output: object
        Converted input item
    """
    if isinstance(item, str) and item == "__none__":
        output = None
    elif isinstance(item, bytes) and item == b"__none__":
        output = None
    elif isinstance(item, (bytes, bytearray)):
        output = item.decode()
    elif isinstance(item, np.ndarray):
        if item.size == 0:
            output = item
        elif "|S" in str(item.dtype) or isinstance(item[0], bytes):
            output = [it.decode() for it in item]
        else:
            output = item
    elif isinstance(item, np.bool_):
        output = bool(item)
    else:
        output = item
    return output


def encode_for_hdf5(key: str, item: Any) -> Any:
    """
    Encode an item to a HDF5 saveable format.

    :param item: object
        Object to be encoded, specific options are provided for Bilby types

    :return output: object
        Input item converted into HDF5 saveable format
    """

    if isinstance(item, np.int_):
        item = int(item)
    elif isinstance(item, np.float_):
        item = float(item)
    elif isinstance(item, np.complex_):
        item = complex(item)
    if isinstance(item, (np.ndarray, int, float, complex, str, bytes)):
        output = item
    elif item is None:
        output = "__none__"
    elif isinstance(item, list):
        if len(item) == 0:
            output = item
        elif isinstance(item[0], (str, bytes)) or item[0] is None:
            output = list()
            for value in item:
                if isinstance(value, str):
                    output.append(value.encode("utf-8"))
                elif isinstance(value, bytes):
                    output.append(value)
                else:
                    output.append(b"__none__")
        elif isinstance(item[0], (int, float, complex)):
            output = np.array(item)
        else:
            raise ValueError(f'Cannot save {key}: {type(item)} type')
    elif isinstance(item, pd.DataFrame):
        output = item.to_dict(orient="list")
    elif inspect.isfunction(item) or inspect.isclass(item):
        output = dict(
            __module__=item.__module__, __name__=item.__name__, __class__=True
        )
    elif isinstance(item, dict):
        output = item.copy()
    elif isinstance(item, tuple):
        output = {str(ii): elem for ii, elem in enumerate(item)}
    elif isinstance(item, datetime.timedelta):
        output = item.total_seconds()
    else:
        raise ValueError(f'Cannot save {key}: {type(item)} type')
    return output


In [110]:
import os
import h5py
import pandas as pd

from typing import Optional


class CompasOutput:
    def __init__(
            self,
            outdir: str,
            Run_Details: Optional[dict] = None,
            BSE_Supernovae: Optional[pd.DataFrame] = None,
            BSE_System_Parameters: Optional[pd.DataFrame] = None,
            BSE_Common_Envelopes: Optional[pd.DataFrame] = None,
            BSE_RLOF: Optional[pd.DataFrame] = None,
    ):
        """

        :param outdir:
        :param Run_Details: Configs for the run
        :param BSE_Supernovae: pd.DataFrame of supernovae
        :param BSE_System_Parameters: pd.DataFrame of system parameters
        """
        self.outdir = outdir
        self.Run_Details = Run_Details
        self.BSE_Supernovae = BSE_Supernovae
        self.BSE_System_Parameters = BSE_System_Parameters
        self.BSE_Common_Envelopes = BSE_Common_Envelopes
        self.BSE_RLOF = BSE_RLOF
        self.number_of_systems = self.Run_Details['number-of-systems']
        self.detailed_output_exists = True if self.Run_Details["detailed-output"] == 1 else False

    def __getitem__(self, item):
        return self.get_binary(index=item)
    
    def get_binary(self, index=None, seed=None):
        all_seeds = self.BSE_System_Parameters.SEED 
        if index is not None:
            seed = all_seeds.iloc[index]
        else:
            index = self.BSE_System_Parameters[all_seeds==seed].index[0]
        data = {}
        if self.detailed_output_exists:
            det_fn = os.path.join(self.outdir, "Detailed_Output", f"BSE_Detailed_Output_{index}.h5")
            data["detailed_output"] = load_compas_detailed_output(det_fn)
        for key in self.__dict__:
            val = self.__dict__[key]
            if isinstance(val, pd.DataFrame):
                val = val[val['SEED']==seed]
                if len(val)==0:
                    data[key] = None
                elif len(val) > 1:
                    raise ValueError("Data for two binaries with same key")
                else:
                    data[key] = val.to_dict('records')[0]
                
        return data

    @classmethod
    def from_hdf5(cls, outdir):
        """
        Loads a COMPAS output file from HDF5 format.

        :param outdir: the directory where the output files are written
        :return: a COMPASOutput object
        """

        filename = os.path.join(outdir, 'COMPAS_Output.h5')
        with h5py.File(filename, "r") as ff:
            data = recursively_load_dict_contents_from_group(ff, '/')
        run_details = data['Run_Details']
        for k in data.keys():
            data[k] = pd.DataFrame(data[k])
        data['Run_Details'] = pd.DataFrame(run_details).to_dict('records')[0]
        data['outdir'] = outdir
        return cls(**data)
    
    def __repr__(self):
        rep = ["COMPAS OUTPUT"]
        for k,v in self.__dict__.items():
            att_rep = f"--{k}: {v},"
            if isinstance(v, str):
                pass
            elif hasattr(v, "__len__"):
                att_rep = f"--{k}: {len(v)},"
            if hasattr(v, "shape"):
                att_rep = f"--{k}: shape {v.shape},"
            rep.append(att_rep)
        return "\n".join(rep)
    
    
def load_compas_detailed_output(filename):
    with h5py.File(filename, "r") as ff:
        data = recursively_load_dict_contents_from_group(ff, '/')
    return pd.DataFrame(data)


compas_out = CompasOutput.from_hdf5(outdir='/Users/avaj0001/Documents/projects/compas_dev/COMPAS/src/COMPAS_Output')
compas_out.get_binary(seed=1667215001)



{'detailed_output':            Age(1)    Age(2)  Ang_Momentum(1)  Ang_Momentum(2)  \
 0    0.000000e+00  0.000000         0.000398         0.000062   
 1    3.955913e-05  0.000040         0.000398         0.000062   
 2    3.955913e-05  0.000040         0.000398         0.000062   
 3    3.955913e-05  0.000040         0.000398         0.000062   
 4    3.955915e-05  0.000040         0.000398         0.000062   
 ..            ...       ...              ...              ...   
 672  9.580939e-07  4.096877         0.000005         0.000052   
 673  9.580939e-07  4.096877         0.016978         0.177946   
 674  9.580939e-07  4.096877         0.016978         0.177946   
 675  9.580939e-07  4.096877         0.016978         0.177946   
 676  9.580939e-07  4.096877         0.016978         0.177946   
 
      Ang_Momentum_Total  Dominant_Mass_Loss_Rate(1)  \
 0           2041.820645                           0   
 1           2041.820645                           5   
 2           2041.8

In [105]:
compas_out.BSE_System_Parameters[compas_out.BSE_System_Parameters['SEED']==1667215002].index

Int64Index([1], dtype='int64')

In [90]:
compas_out[1667215001]

OSError: Unable to open file (file read failed: time = Tue Nov  1 00:37:56 2022
, filename = '/Users/avaj0001/Documents/projects/compas_dev/COMPAS/src/COMPAS_Output', file descriptor = 72, errno = 21, error message = 'Is a directory', buf = 0x7ff7bf233c88, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)

In [78]:
compas_out.BSE_RLOF.shape

(4, 39)

In [69]:
compas_out = CompasOutput.from_hdf5(outdir='/Users/avaj0001/Documents/projects/compas_dev/compas_ml_surrogate/tests/test_data/COMPAS_Output/')
compas_out.


COMPAS OUTPUT
--outdir: /Users/avaj0001/Documents/projects/compas_dev/compas_ml_surrogate/tests/test_data/COMPAS_Output/,
--Run_Details: 432 rows,
--BSE_Supernovae: 5 rows,
--BSE_System_Parameters: 5 rows,
--BSE_Common_Envelopes: None,
--BSE_RLOF: None,
--number_of_systems: 5,
--detailed_output_exists: False,

In [29]:
import h5py
import os 
import pandas as pd
from pprint import pprint

f = '/Users/avaj0001/Documents/projects/compas_dev/COMPAS/src/COMPAS_Output/COMPAS_Output.h5'

with h5py.File(f, "r") as ff:
    data = recursively_load_dict_contents_from_group(ff, '/')

for k in data:
    print(k)
    pprint(data[k])

BSE_Common_Envelopes
{'BE_Fixed(1)': array([2.83480353e+49, 1.55345259e+48, 0.00000000e+00, 2.20902768e+48]),
 'BE_Fixed(2)': array([0.00000000e+00, 0.00000000e+00, 2.07600249e+48, 0.00000000e+00]),
 'BE_Kruckow(1)': array([2.36398845e+49, 6.79832242e+47, 0.00000000e+00, 6.01654911e+47]),
 'BE_Kruckow(2)': array([0.00000000e+00, 0.00000000e+00, 1.18737497e+47, 0.00000000e+00]),
 'BE_Loveridge(1)': array([1.57345242e+50, 2.35015544e+46, 0.00000000e+00, 1.77180950e+47]),
 'BE_Loveridge(2)': array([0.00000000e+00, 0.00000000e+00, 2.06787668e+47, 0.00000000e+00]),
 'BE_Loveridge_Winds(1)': array([2.26292173e+50, 2.46758349e+46, 0.00000000e+00, 1.77857451e+47]),
 'BE_Loveridge_Winds(2)': array([0.00000000e+00, 0.00000000e+00, 2.06836624e+47, 0.00000000e+00]),
 'BE_Nanjing(1)': array([5.66960705e+49, 3.02583755e+46, 0.00000000e+00, 1.12584019e+46]),
 'BE_Nanjing(2)': array([0.0000000e+00, 0.0000000e+00, 1.3372184e+47, 0.0000000e+00]),
 'Binding_Energy<CE(1)': array([5.66960705e+49, 3.0258375

 'quiet': array([0], dtype=uint8),
 'quiet-Derivation': ['DEFAULT_USED '],
 'random-seed': array([0], dtype=uint64),
 'random-seed-Derivation': ['DEFAULT_USED '],
 'remnant-mass-prescription': ["'FRYER2012'"],
 'remnant-mass-prescription-Derivation': ['DEFAULT_USED '],
 'retain-core-mass-during-caseA-mass-transfer': array([0], dtype=uint8),
 'retain-core-mass-during-caseA-mass-transfer-Derivation': ['DEFAULT_USED '],
 'revised-energy-formalism-nandez-ivanova': array([0], dtype=uint8),
 'revised-energy-formalism-nandez-ivanova-Derivation': ['DEFAULT_USED '],
 'rlof-printing': array([1], dtype=uint8),
 'rlof-printing-Derivation': ['DEFAULT_USED '],
 'rotational-frequency': array([0.]),
 'rotational-frequency-1': array([0.]),
 'rotational-frequency-1-Derivation': ['DEFAULT_USED '],
 'rotational-frequency-2': array([0.]),
 'rotational-frequency-2-Derivation': ['DEFAULT_USED '],
 'rotational-frequency-Derivation': ['DEFAULT_USED '],
 'rotational-velocity-distribution': ["'ZERO'"],
 'rotatio

In [33]:
pd.DataFrame(data['Run_Details']).to_dict('records')[0]

[{'Actual-Random-Seed': 1667215001,
  'Actual-Random-Seed-Derivation': 'CALCULATED   ',
  'COMPAS-Version': '02.33.01',
  'COMPAS-Version-Derivation': 'CALCULATED   ',
  'Clock-Time': 10.862766,
  'Clock-Time-Derivation': 'CALCULATED   ',
  'Objects-Created': 5,
  'Objects-Created-Derivation': 'CALCULATED   ',
  'Objects-Requested': 5,
  'Objects-Requested-Derivation': 'CALCULATED   ',
  'PISN-lower-limit': 60.0,
  'PISN-lower-limit-Derivation': 'DEFAULT_USED ',
  'PISN-upper-limit': 135.0,
  'PISN-upper-limit-Derivation': 'DEFAULT_USED ',
  'PPI-lower-limit': 35.0,
  'PPI-lower-limit-Derivation': 'DEFAULT_USED ',
  'PPI-upper-limit': 60.0,
  'PPI-upper-limit-Derivation': 'DEFAULT_USED ',
  'Run-End': 'Mon Oct 31 22:16:52 2022\n',
  'Run-End-Derivation': 'CALCULATED   ',
  'Run-Start': 'Mon Oct 31 22:16:41 2022\n',
  'Run-Start-Derivation': 'CALCULATED   ',
  'Wall-Time': '0000:00:11',
  'Wall-Time-Derivation': 'CALCULATED   ',
  'actual-output-path': '/Users/avaj0001/Documents/project

In [25]:
import h5py
import os 
import pandas as pd

f = '/Users/avaj0001/Documents/projects/compas_dev/COMPAS/src/COMPAS_Output/Detailed_Output/BSE_Detailed_Output_0.h5'

with h5py.File(f, "r") as ff:
    data = recursively_load_dict_contents_from_group(ff, '/')

for k in data:
    print(k)
    display(pd.DataFrame(data[k]))

Age(1)


,0
0,0.000000e+00
1,3.955913e-05
2,3.955913e-05
3,3.955913e-05
4,3.955915e-05
...,...
672,9.580939e-07
673,9.580939e-07
674,9.580939e-07
675,9.580939e-07


Age(2)


,0
0,0.000000
1,0.000040
2,0.000040
3,0.000040
4,0.000040
...,...
672,4.096877
673,4.096877
674,4.096877
675,4.096877


Ang_Momentum(1)


...

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
672,0.0
673,0.0
674,0.0
675,0.0


dmWinds(1)


,0
0,0.000000
1,0.000000
2,0.000000
3,-0.000062
4,-0.000062
...,...
672,0.000000
673,0.000000
674,0.000000
675,0.000000


dmWinds(2)


,0
0,0.000000e+00
1,0.000000e+00
2,0.000000e+00
3,-1.391796e-07
4,-1.391796e-07
...,...
672,0.000000e+00
673,0.000000e+00
674,0.000000e+00
675,0.000000e+00


In [21]:
! ls /Users/avaj0001/Documents/projects/compas_dev/COMPAS/src/COMPAS_Output/Detailed_Output/

BSE_Detailed_Output_0.h5 BSE_Detailed_Output_2.h5 BSE_Detailed_Output_4.h5
BSE_Detailed_Output_1.h5 BSE_Detailed_Output_3.h5
